In [1]:
project_id = 'cp100-183410'
namespace = 'myAnimeList'
kind = 'anime'

# Datastore Client API

### Read json

In [2]:
import json
anime_json = json.load(open('anime.json','r'))

In [18]:
anime_json['myanimelist']['myinfo']

{'user_completed': '87',
 'user_days_spent_watching': '36.19',
 'user_dropped': '14',
 'user_id': '1320541',
 'user_name': 'neurotichl',
 'user_onhold': '6',
 'user_plantowatch': '10',
 'user_watching': '3'}

In [9]:
anime_sample = anime_json['myanimelist']['anime'][0]
anime_sample

{'my_finish_date': '0000-00-00',
 'my_id': '0',
 'my_last_updated': '1338183572',
 'my_rewatching': '0',
 'my_rewatching_ep': '0',
 'my_score': '5',
 'my_start_date': '0000-00-00',
 'my_status': '4',
 'my_tags': None,
 'my_watched_episodes': '156',
 'series_animedb_id': '20',
 'series_end': '2007-02-08',
 'series_episodes': '220',
 'series_image': 'https://myanimelist.cdn-dena.com/images/anime/13/17405.jpg',
 'series_start': '2002-10-03',
 'series_status': '2',
 'series_synonyms': 'NARUTO; Naruto',
 'series_title': 'Naruto',
 'series_type': '1'}

---

### Import datastore client library
https://cloud.google.com/datastore/docs/datastore-api-tutorial

In [3]:
from google.cloud import datastore

- Define datastore client

In [4]:
ds = datastore.Client(project_id, namespace = namespace)

- Upload data into datastore

(1) Create a key

In [5]:
key = ds.key(kind)

 (2) Create an entity

In [6]:
entity = datastore.Entity(key=key)

(3) Fill entity with values

In [10]:
entity.update(anime_sample)

(4) Upload to datastore using client

In [12]:
ds.put(entity)

![Datastore](https://github.com/neurotichl/GCP/raw/master/img/datastore_preview2.png)

- Get data from datastore

In [15]:
query = ds.query(kind=kind)
list(query.fetch(1))

[<Entity('anime', 5749328048029696) {'my_score': '5', 'my_last_updated': '1338183572', 'my_start_date': '0000-00-00', 'my_status': '4', 'my_rewatching_ep': '0', 'series_synonyms': 'NARUTO; Naruto', 'series_type': '1', 'my_finish_date': '0000-00-00', 'series_status': '2', 'my_rewatching': '0', 'series_episodes': '220', 'my_tags': None, 'series_image': 'https://myanimelist.cdn-dena.com/images/anime/13/17405.jpg', 'my_id': '0', 'series_end': '2007-02-08', 'series_start': '2002-10-03', 'series_animedb_id': '20', 'my_watched_episodes': '156', 'series_title': 'Naruto'}>]

In [16]:
entity_value = list(query.fetch(1))[0]
print('key: ',entity_value.key, end='\n\n')
print('values: ',entity_value.items())

key:  <Key('anime', 5749328048029696), project=cp100-183410>

values:  dict_items([('my_score', '5'), ('my_last_updated', '1338183572'), ('my_start_date', '0000-00-00'), ('my_rewatching', '0'), ('my_rewatching_ep', '0'), ('my_watched_episodes', '156'), ('series_type', '1'), ('my_finish_date', '0000-00-00'), ('series_status', '2'), ('my_status', '4'), ('series_episodes', '220'), ('my_tags', None), ('series_image', 'https://myanimelist.cdn-dena.com/images/anime/13/17405.jpg'), ('my_id', '0'), ('series_end', '2007-02-08'), ('series_start', '2002-10-03'), ('series_animedb_id', '20'), ('series_synonyms', 'NARUTO; Naruto'), ('series_title', 'Naruto')])


- Update an entity

In [17]:
key = ds.key(kind, entity_value.key.id) # Put the entity unique key value in client's key
entity = datastore.Entity(key=key)
entity.update({k:v if k != 'my_score' else '6.5' for k,v in anime_sample.items()})
ds.put(entity)

![Datastore](https://github.com/neurotichl/GCP/raw/master/img/datastore_preview3.png)

- Delete an entity

In [18]:
ds.delete(key)

![Datastore](https://github.com/neurotichl/GCP/raw/master/img/datastore_preview4.png)

---

##### Add all json to datastore

In [19]:
for data in anime_json['myanimelist']['anime']:
  key = ds.key(kind)
  entity = datastore.Entity(key=key)
  entity.update(data)
  ds.put(entity)

- Query with filter

In [71]:
query = ds.query(kind=kind)
query.add_filter('my_score', '=', '10') # Add Filter
query_result = query.fetch()
list([anime['series_title'] for anime in query_result])

['Re:Zero kara Hajimeru Isekai Seikatsu',
 'Cardcaptor Sakura',
 'Mononoke',
 'Fate/Zero',
 'Fate/Zero 2nd Season',
 'Shingeki no Kyojin',
 'Tenkuu no Shiro Laputa',
 'Lovely★Complex',
 'Hunter x Hunter (2011)']

![Datastore](https://github.com/neurotichl/GCP/raw/master/img/datastore_preview5.png)